In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from utils import get_data
print(torch.cuda.is_available())

True


In [2]:
path = './prosperLoanData.csv'
k=9
data = get_data(path, k=k)

Training acc: 0.9996
Training conf matrix: [[17700     0]
 [    9  5320]]


In [3]:
data.head()

,LoanKey,MemberKey,Term,BorrowerRate,LenderYield,EstimatedEffectiveYield,EstimatedLoss,EstimatedReturn,ProsperRating (numeric),ProsperScore,...,InvestmentFromFriendsAmount,Investors,finished,risk,EmploymentStatus_Full-time,EmploymentStatus_Not employed,EmploymentStatus_Other,EmploymentStatus_Part-time,EmploymentStatus_Retired,EmploymentStatus_Self-employed
15,i12,u12,-0.574149,1.681937,1.719719,1.825192,1.880974,0.980611,-1.888061,-0.449555,...,-0.034066,-0.607892,1,0.999076,False,False,True,False,False,False
23,i18,u18,-0.574149,0.656200,0.696186,0.682534,0.629259,0.328283,-0.682340,-0.449555,...,-0.034066,-0.402626,1,0.999076,False,False,False,False,False,False
26,i21,u21,1.474971,1.183954,1.155581,1.197472,0.955794,0.987302,-1.285200,-1.296151,...,-0.034066,-0.330783,1,0.001322,False,False,False,False,False,False
27,i22,u22,-0.574149,-1.372269,-1.483500,-1.570135,-1.253756,-1.153675,1.126242,1.243637,...,-0.034066,0.531334,1,0.001322,True,False,False,False,False,False
30,i25,u25,-0.574149,-1.265365,-1.483500,-1.141267,-0.883683,-1.458095,1.126242,1.243637,...,-0.034066,-0.402626,1,0.001322,False,False,False,False,False,False


In [8]:
# Create a matrix of investor-loan interactions
member_mapping = {member: idx for idx, member in enumerate(data['MemberKey'].unique())}
loan_mapping = {loan: idx for idx, loan in enumerate(data['LoanKey'].unique())}

n = len(member_mapping)
m = len(loan_mapping)
A = np.zeros((n, m), dtype=int)

# Fill the matrix using the mappings
for _, row in data.iterrows():
    member_idx = member_mapping[row['MemberKey']]
    loan_idx = loan_mapping[row['LoanKey']]
    A[member_idx, loan_idx] = 1

In [12]:
# Create a matrix of loan-risk interactions
risk_mapping = {risk: idx for idx, risk in enumerate(data['risk'].unique())}
p = len(risk_mapping)

P = np.zeros((m, p), dtype=int)

# Fill the matrix using the mappings
for _, row in data.iterrows():
    loan_idx = loan_mapping[row['LoanKey']]
    risk_idx = risk_mapping[row['risk']]
    P[loan_idx, risk_idx] = 1

# Create a matrix of loan-returns interactions
return_mapping = {ret: idx for idx, ret in enumerate(data['LenderYield'].unique())}
q = len(return_mapping)

Q = np.zeros((m, q), dtype=int)

# Fill the matrix using the mappings
for _, row in data.iterrows():
    loan_idx = loan_mapping[row['LoanKey']]
    return_idx = return_mapping[row['LenderYield']]
    Q[loan_idx, return_idx] = 1

In [14]:
U = data['MemberKey'].unique()
I = data['LoanKey'].unique()
S = data['risk'].unique()
T = data['LenderYield'].unique()

In [15]:
S

array([0.99907587, 0.00132206, 0.9496424 , 0.74269035, 0.86835363,
       0.59295706, 0.24902838, 0.09793692, 0.43852724])